In [48]:
import subprocess
import os
import netifaces
import pprint
from netifaces import interfaces
import winreg as wr
import re

# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(netifaces.interfaces())

pattern = r'^10'

#定义获取Windows系统网卡接口的在注册表的键值的函数
def get_name_AND_ip():
    #获取所有网络接口卡的键值
    id = interfaces()
    #存放网卡键值与键值名称的字典
    key_name = {}
    try:
        #建立链接注册表，"HKEY_LOCAL_MACHINE"，None表示本地计算机
        reg = wr.ConnectRegistry(None,wr.HKEY_LOCAL_MACHINE)
        # 打开r'SYSTEM\CurrentControlSet\Control\Network\{4d36e972-e325-11ce-bfc1-08002be10318}'，固定的
        reg_key = wr.OpenKey(reg , r'SYSTEM\CurrentControlSet\Control\Network\{4d36e972-e325-11ce-bfc1-08002be10318}')
    except :
        return ('路径出错或者其他问题，请仔细检查')


    for i in id:
        try:
            #尝试读取每一个网卡键值下对应的Name
            reg_subkey = wr.OpenKey(reg_key , i + r'\Connection')
            #如果存在Name，写入key_name字典
            key_name[wr.QueryValueEx(reg_subkey , 'Name')[0]] = i
            # print(wr.QueryValueEx(reg_subkey , 'Name')[0])
        except FileNotFoundError:
            pass
    # print('所有接口信息字典列表： ' + str(key_name) + '\n')
    

#     for name, key in key_name.items():
#         print(name)
#         try:
#             pp.pprint(netifaces.ifaddresses(key)[netifaces.AF_INET][0]['addr'])
#         except KeyError:
#             print("NULL")
#             pass
#         print("---------------------------------")

    for name, key in key_name.items():
        
        try:
            ip = netifaces.ifaddresses(key)[netifaces.AF_INET][0]['addr']       
#             print(f"ip=：{ip}")
            if re.search(pattern, ip):                
                break
        except KeyError:
            pass 
        
    if re.search(pattern, ip): 
        print(f"查询匹配的网卡名称：{name}")
        print(f"对应的IP：{ip}")
#         cmd = "ipconfig"
        cmd = "netsh interface ip set dnsservers name=\"%s\" source=static addr=2.2.2.2" % name
        
#         timeout=18000
        try:
            sp = subprocess.Popen(
            cmd,
            shell=True
#             stdout=subprocess.PIPE,
#             stderr=subprocess.PIPE
            )
            print("更改主DNS[PID] %s: %s" % (sp.pid, cmd))
#             cmd = "netsh interface ip add dnsservers name=\"%s\" addr=10.10.10.10 > nul" % name
#             sp2 = subprocess.Popen(
#             cmd,
#             shell=True
#             )
#             print("更改备DNS[PID] %s: %s" % (sp2.pid, cmd))
#             sp.wait(timeout=timeout)

        except Exception as e:
            raise e
        
#         os.system("netsh interface ip set dnsservers name=%s source=static addr=1.1.1.1" % '以太网')
#         proc = subprocess.Popen("netsh interface ip set dnsservers name=%s source=static addr=1.1.1.1" % '以太网') 
#         time.sleep(5) 
#         pobj = psutil.Process(proc.pid) 
        print(f"更改主备DNS成功！")
#         netsh interface ip set dnsservers name="%a%" source=static addr=10.157.160.89
    else:
        name = 'No matching IP was found'
        ip = ''
    data =[ [name],[ip]]
    
    
#     with open("data.txt","w") as f:                                                   #设置文件对象
#         for i in data:                                                                 #对于双层列表中的数据
#             i = str(i).strip('[').strip(']').replace(',','').replace('\'','')+'\n'  #将其中每一个列表规范化成字符串
#             f.write(i)    
    result = (name, ip)
    return result


a = get_name_AND_ip()
# print(f"查询匹配的网卡：{a}")
# os.system("ipconfig")
os.system("pause")


查询匹配的网卡名称：以太网
对应的IP：10.18.48.9
更改主DNS：1.1.1.1 成功！


0